<a href="https://colab.research.google.com/github/matsunagalab/tutorial_analyzingMDdata/blob/main/01_mdtraj_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDTraj basics

## MDTrajのインストールとデータのダウンロード

In [1]:
# Colab environment で mdtraj をインストールする
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()
!mamba install mdtraj

# T4 LysozymeのMDデータの取得
# 溶媒は削除済みの wildtype.pdb と wildtype.dcd をダウンロードする。
!gdown 1Lu22Z7ARKSws77sBaYr84iCcQsnTHId9
!gdown 1Eh0SSSxgAmv7LI2NvAY43tLMLI63lSa-

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:19
🔁 Restarting kernel...

Looking for: ['mdtraj']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   7%
conda-forge/noarch     3%[+] 0.3s
conda-forge/linux-64  15%
conda-forge/noarch    16%[+] 0.4s
conda-forge/linux-64  23%
conda-forge/noarch    35%[+] 0.5s
conda-forge/linux-64  31%
conda-forge/noarch    45%[+] 0.6s
conda-forge/linux-64  36%
conda-forge/noarch    54%[+] 0.7s
conda-forge/linux-64  40%
conda-forge/noarch    64%[+] 0.8s
conda-forge/linux-64  44%
conda-forge/noarch    84%[+] 0.9s
conda-forge/linux-64  52%
conda-forge/noarch    93%conda-forge/noarch                                
[+] 1.0s
conda-forge/linux-64  57%[+] 1.1s
conda-forge/linux-64  65%[+] 1.2s
conda-forge/linux-64  74%[+] 1.3s
conda-forge/linux-64  90%con

## MDTrajを使ってみる

以下の文書は MDTrajのドキュメント https://www.mdtraj.org/1.9.7/examples/introduction.html を主に日本語訳したものです。

ディスクからトラジェクトリをロードして始めます。MDTrajは自動的にファイル拡張子を解析し、適切なローダーを使用します。
Topology情報を含まないファイルをロードするには、Gromacs XTCファイルのようなもの、
Topologyを記述する何か、たとえばPDBファイル、をtopキーワード引数で提供する必要があります。

In [2]:
import mdtraj as md
import numpy as np

# DCDファイルからトラジェクトリを読み込み、対応するPSFトポロジーファイルを使用する
# トラジェクトリオブジェクト t が作成される
t = md.load('/content/wildtype.dcd', top='/content/wildtype.pdb')

# t の中身を出力する
print(t)

<mdtraj.Trajectory with 10000 frames, 2643 atoms, 164 residues, and unitcells>


Trajectoryのフレームの一部だけに興味がある場合は、それをスライスすることができます。

In [7]:
# 最初の10フレームを見てみましょう
print(t[:100:2])

<mdtraj.Trajectory with 50 frames, 2643 atoms, 164 residues, and unitcells>


In [8]:
# それとも最後のフレームを見る？
print(t[-1])

<mdtraj.Trajectory with 1 frames, 2643 atoms, 164 residues, and unitcells>


Trajectoryオブジェクトには多くの情報があります。自明なのは
デカルト座標です。それらはxyzの下のnumpy配列として格納されています。全ての
座標の長さはナノメートルで保存されています。時間単位
はピコ秒です。角度はdegree (radianではない) で保存されています。

Trajectoryオブジェクトが持っているプロパティやメソッドは[ドキュメント](https://www.mdtraj.org/1.9.7/api/generated/mdtraj.Trajectory.html#)で確認することができます。

In [ ]:
t.xyz.shape

(6768, 2643, 3)

In [14]:
np.mean(t.xyz)

3.321987

In [15]:
# 最初の10フレームのシミュレーション時間（ピコ秒）
t.time[0:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [17]:
# 最後のフレームの単位セルの長さは？（ナノメートル）
t.unitcell_lengths[-1]

array([6.9423013, 6.9423013, 6.9423013], dtype=float32)

Trajectoryをディスクに戻すのは簡単です。

In [18]:
t.save('trajectory.h5')

In [ ]:
# hdf5形式はファイル内にトポロジを保存するので便利です
t[::2].save('halftraj.h5')

In [19]:
# フォーマットは拡張子に基づいて解析されます、またはフォーマット固有のsaveメソッドを呼び出すこともできます
t[:10].save_dcd('first-ten-frames.dcd')

TrajectoryオブジェクトにはTopologyオブジェクトへが含まれており、便利に利用することが出来ます。例えば、alpha carbon のみを含むトラジェクトリのコピーを保存したい場合、以下のようにできます。

In [21]:
t.topology.atoms

<generator object Topology.atoms at 0x7bc58305dfc0>

In [22]:
atoms_to_keep = [a.index for a in t.topology.atoms if a.name == 'CA']
t.restrict_atoms(atoms_to_keep) # この操作はトラジェクトリオブジェクトを変化させます
t.save('CA-only.h5')

Topogyオブジェクトが持つプロパティやメソッドは[ドキュメント](https://www.mdtraj.org/1.9.7/api/generated/mdtraj.Topology.html)で確認することができます。